# Updating the Quezon City Government Leadership and Development Database System for the Automated Production of the Trainings Scorecard
### by Ian Salig U. Batangan

## Project Scope


This is an internship output for the Strategic Human Resource Unit(SHRU) of the Quezon City Government.

For compliance of the status of PRIME HRM by the Civil Service Commission(CSC), SHRU needs to update their data monitoring for the current feedback system for the trainings they dole out, to upskill and certify current employees, in order to efficiently monitor the effect of the trainings to the employees and also measure the metrics needed for the status compliance.

The challenge is to design an add-on the current system that can automate the cleaning of data and to create a google sheet that can recall and store the needed data.

## Methodology


SHRU's current system is heavily based on google sheets, primarily since it is a free alternative to a cloud database where a person without any coding expertise can manipulate and collate data, and can easily share with another individual without a paywall.

The primary system used is done via downloading the training registration, in xlxs format, and post-training, in csv format, files on the participants from a third-party application. It is then collated manually and inserted to a data visualization google sheet. This does not accurately show when the data is looked as a whole, and the visualizations can only show per training and done by manually creating pivot tables. 

In order to streamline the process and reduce the human error, a two-pronged approach is created. A python file that will clean and collate the data, and an automated Google Sheet that can sort which data to show relevant data.

### Manual Part

The data should be downloaded from the third party application, then renamed to their specified training code, batch number, and what type of data (registration or post training).

|  | Format | Example|
|:--------:|:--------:|:--------:|
|  Registration Data  |  [TRAINING CODE][BATCH NUMBER]_Reg.xlsx  |  WETCT3_Reg.xlsx  |
|  Post-Training Data |  [TRAINING CODE][BATCH NUMBER]_Post.csv   |  WETCT3_Post.csv    |




It is then placed in a folder containing the python program and a folder named "MergedFiles2". 

### Python Code

Since the datasets can contain up to 500 participants per training, manual encoding/collating of the data is gruelling and time consuming. A python code is created to be able to select and collate the data needed for data visualizaation. The Python Code is also created in Python3.13 environment but no special recent update is used so it should work at any Python3 environment.

The code starts with an inbuilt installer of the 3rd party libraries, not in the Python Standard Libraries

In [ ]:
def pip(): #install needed 3rd party libraries
    import sys
    import subprocess
    # implement pip as a subprocess:
    packagename=('pandas','pathlib','openpyxl')
    for i in packagename:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', i])
    # process output with an API in the subprocess module:
    reqs = subprocess.check_output([sys.executable, '-m', 'pip', 'freeze'])
    installed_packages = [r.decode().split('==')[0] for r in reqs.split()]
    print(installed_packages)
#pip()

The libraries used within the python code is then imported

In [ ]:
#for data manipulation library, also needs openpyxl sub library of pandas, pandas library
import pandas as pd

#string manipulation, regular expressions library
import re

#for finding file path, glob library and pathlib library
import glob
from pathlib import Path

#### Custom Merge Function

Its primary purpose is to left join the post training dataset and the relevant data from the registrants dataset. The function takes in two variables.

In [ ]:
def merge(variable_name,root_path):

The path of the file to be used is defined by the root_path variable and the variable_name by using f-string.

In [ ]:
#determining path
path_reg = f"{root_path}/{variable_name}_Reg.xlsx"
path_post = f"{root_path}/{variable_name}_Post.csv"

The relevant data is then turned into a dataframe so that data manipulation using pandas is possible. THe registration dataset is from a .xlsx file and the post training dataset is from a .csv file.

In [ ]:
#reads raw excel and csv
df_reg = pd.read_excel(path_reg)
df_post = pd.read_csv(path_post)

To prevent duplicate columns with different capitalizations all the columns are uppercased.

In [ ]:
df_reg.columns = map(str.upper, df_reg.columns)
df_post.columns = map(str.upper, df_post.columns)

The columns is then made into a list.

In [ ]:
df_reg_columns = df_reg.columns
df_post_columns = df_post.columns 

There are some trainings that did not require the participants assessment scores. But since python returns an error value when we try to pull this data, we tag the dataset if they do not have an assessment score.

In [ ]:
   
#checking the list if the data set has assessment
if 'PRE-ASSESSMENT TOTAL' not in df_reg_columns or 'QTOTAL' not in df_post_columns:check_pre=1
else: check_pre= 0

Since the dataset is cluttered with uneeded data, the extrenous data is sorted out of the list of columns using list comprehension.

In [ ]:
#mass removal of extrenous data
df_reg_cleaned = [e for e in df_reg_columns if "PRE-ASSESSMENT" not in e and 'EMAIL ADDRESS' not in e and  \
                          'NICKNAME' not in e and 'ENDORSEMENT LETTER' not in e and 'CSC UPLOADED' not in e \
                        and 'DATE ANSWERED' not in e and 'EXPECTED OUTCOMES'not in e and 'DATA PRIVACY CONSENT' \
                            not in e and 'CONTACT NUMBER' not in e]

In [ ]:
#using re library to exlude columns with Q in name in specific cases
df_post_cleaned = [e for e in df_post_columns if not re.match(re.compile('Q.+-' ) , e) and not re.match(re.compile('Q..' ) , e)and not re.match(re.compile('Q.' ) , e) \
                        and 'EMAIL ADDRESS' not in e and 'NICKNAME' not in e and \
                        'ENDORSEMENT LETTER' not in e and 'CSC UPLOADED' not in e and 'DATE ANSWERED' not in e and \
                            'EXPECTED OUTCOMES'not in e and 'DATA PRIVACY CONSENT'not in e and 'CONTACT NUMBER' not in e]
        

Since the previous step was a total wipe we add the crucial assesment scoring data back into the list of columns.

In [ ]:
#adds back the needed column since mass deletion was done previously
if check_pre == 0:
    df_reg_cleaned = df_reg_cleaned+['PRE-ASSESSMENT TOTAL']
    df_post_cleaned = df_post_cleaned+ ['QTOTAL']

Since max score is not specified, the number of columns with "Q..." is counted and the number of elements is the number of questions therefore the max available score since the scores are all weighted equally.

In [ ]:
 #finding maximum score
Qmax_list = [e for e in df_post_columns if re.match(re.compile('Q.+-' ) , e) ]
max_score=len(Qmax_list)

With the column names cleaned, the relevant data can be extracted from the data frames. The max score and training code of the data is also added to the dataframe.

In [ ]:
#retrieving specific column names
df_reg = df_reg.loc[:,df_reg_cleaned]
df_post = df_post.loc[:,df_post_cleaned]
df_post= df_post.assign(Maximum_Assesment_Score=max_score,Training_Code=variable_name)

The joining of the datasets is done with the merge command of pandas. Since some datasets have full names instead of spearated names, an if-else case is created to accomodate different types of datasets. The joining is done with the names, designation, section department, employment type, and sex of the training participant. The dataset sometimes mutates so the drop duplicates command is done after. It is then saved at MergedFiles2 as a .csv file.

In [ ]:
#Left Join of post and reg data and drops duplicates
        #checking if data set has full name if not full name is created
if 'FULL NAME' in df_post_columns and 'FULL NAME' in df_reg_columns:
        df_merge= df_reg.merge(df_post,left_on=['FULL NAME',"DESIGNATION/POSITION","DIVISION/ SECTION",\
                                                    'DEPARTMENT/ OFFICE/ UNIT/ TASK FORCE','EMPLOYMENT TYPE','SEX'],\
                                                        right_on=['FULL NAME',"DESIGNATION/POSITION","DIVISION/ SECTION",\
                                                    'DEPARTMENT/ OFFICE/ UNIT/ TASK FORCE','EMPLOYMENT TYPE','SEX']).drop_duplicates()
else:
        df_merge= df_reg.merge(df_post,left_on=["LAST NAME","FIRST NAME","MIDDLE INITIAL","DESIGNATION/POSITION",\
                                                    "DIVISION/ SECTION",'DEPARTMENT/ OFFICE/ UNIT/ TASK FORCE','EMPLOYMENT TYPE','SEX'],\
                                                            right_on=["LAST NAME","FIRST NAME","MIDDLE INITIAL","DESIGNATION/POSITION","DIVISION/ SECTION",\
                                                        'DEPARTMENT/ OFFICE/ UNIT/ TASK FORCE','EMPLOYMENT TYPE','SEX']).drop_duplicates()
        #generate Full Name
        df_merge['FULL NAME'] = df_merge["FIRST NAME"]+ " " + df_merge["MIDDLE INITIAL"] + " "+ df_merge["LAST NAME"]
        
    #saves the merged data to a csv file
        
df_merge.to_csv(f"{root_path}/MergedFiles/{variable_name}_merged.csv")

#### Main Function


First we define the path of the file

In [ ]:
#finding file path
root_path=Path.cwd()
print(f'File path is {root_path}')

Using the file names and glob we can define the number of trainings and a list of the trainings is created by looking at the more crucial post-training data

In [ ]:
#checks current folder for all the trainings with post assesment and makes a list of the unique trainings
path_list=glob.glob(f"*_Post.csv")

An empty list and an empty dataframe is then created

In [ ]:
training_list=[]
main_df=pd.DataFrame()

Iterating over the list called path_list and removing the last 9 characters in order to isolate the training name and batch number and the number of elements in the list determines the number of trainings in the folder

In [ ]:
for name in path_list: #change into list comprehension
    training_list.append(name[:-9:])
total_files=len(training_list)

Iterating over the training_list as the input for the custom merge function to left join the post database and the registration database where it is temporarily stored and concatinated to the main dataframe.

In [ ]:
 #uses list of trainings to look for the file to merge
i=0
for name in training_list:
    i+=1
    temp_df=merge(name,root_path) #temporary dataframe
    main_df=pd.concat([main_df,temp_df]) #concatenates the data
    print(f'{name} is merged, with a shape {temp_df.shape} \n {i} out of {total_files}')

Now we clean the presentation of the data for the users ease of use and easier data quality checking. First the list of available columns is created.

In [ ]:
#Re-arranging the Data Columns
main_df_columns= main_df.columns

Isolating the relevant participant data. So that it can be seen at the start of the dataset.

In [ ]:
main_df_columns_tag=["ID",'Training_Code',"FULL NAME","LAST NAME","FIRST NAME","MIDDLE INITIAL",\
                          "DESIGNATION/POSITION","DIVISION/ SECTION",'DEPARTMENT/ OFFICE/ UNIT/ TASK FORCE',\
                            'EMPLOYMENT TYPE','SEX','PRE-ASSESSMENT TOTAL', 'QTOTAL', 'Maximum_Assesment_Score']

The remaining data columns can be subdivided into 7 categories(Program Design, Training Materials, Logistics, Expectations, Administration, Comments, and Facilitator), using list comprehension and regular expressions library on the column list, a list is created for each category.

In [ ]:
main_df_columns_design=[e for e in main_df_columns if re.match(re.compile('PROGRAM DESIGN.+' ) , e) ]
    
main_df_columns_trainingmat=[e for e in main_df_columns if re.match(re.compile('TRAINING.+' ) , e)]

main_df_columns_logistics=[e for e in main_df_columns if re.match(re.compile('LOGISTICS.+' ) , e)]

main_df_columns_expectations=[e for e in main_df_columns if re.match(re.compile('EXPECTATION.+' ) , e)]
    
main_df_columns_administration=[e for e in main_df_columns if re.match(re.compile('ADMINISTRATION.+' ) , e)]

main_df_columns_comments=[e for e in main_df_columns if re.match(re.compile('COMMENT.+' ) , e)]

main_df_columns_facilitators=[e for e in main_df_columns if re.match(re.compile('FACILITATOR.+' ) , e)]

An ordered list is created by joining the different categories. Then the extra columns is also grouped so that no data will be lost. 

In [ ]:
main_df_ordered_list= main_df_columns_tag + main_df_columns_design + main_df_columns_trainingmat + main_df_columns_logistics + main_df_columns_expectations + main_df_columns_administration + main_df_columns_comments + main_df_columns_facilitators
    
main_df_columns_others=[e for e in main_df_columns if e not in main_df_ordered_list]

main_df_ordered_list= main_df_columns_tag + main_df_columns_design + main_df_columns_trainingmat + main_df_columns_logistics + main_df_columns_expectations + main_df_columns_administration + main_df_columns_comments + main_df_columns_facilitators + main_df_columns_others

Using the ordered list, the columns of the main dataframe is rearranged.

In [ ]:
main_df= main_df[main_df_ordered_list]

Then the user is updated on the shape of the main dataframe and is exported to the MergeFiles2 folder.

In [ ]:
print((main_df_ordered_list+main_df_columns_others))

    #feedbacka
print('Main DataFrame is Updated')
print(f'The total data shape is {main_df.shape}')
main_df.to_csv(f"{root_path}/MergedFiles/AllConcat.csv") 

main_df.describe()

### Google Sheets

The data that is required to present is for each individual training, and all the trainings in total. Since the sub-categories varies for each training and to future proof the google sheets to an ever-expanding list of subcategories, the Power Query call is based on a dynamic list instead of calling using hard coordinates/columns. 

#### Data Filtering
The All_Concat file is copy and pasted to the google sheets. Basic Power Query is used with a reference to the inbuilt dashboard in order to select which data to show.

In [ ]:
=IF('DASHBOARD V2'!B3="All",QUERY(AllConcat,"SELECT *"),QUERY(AllConcat,"SELECT * WHERE C='"&'DASHBOARD V2'!B3&"'"))

A different sheet draws from the selected training data and filters out columns that dont have data. The first formula just draws the participant data, the second formula draws the filtered data.

In [ ]:
=QUERY(PullSpecificTraining!A1:O)

=FILTER(PullSpecificTraining!P1:HG,LEN(TRIM(QUERY(PullSpecificTraining!P2:HG,,9^9))))

From this filter sheet a columns list made and transposed for easier calling later on. 

In [ ]:
=unique(TRANSPOSE(Filtered!1:1))

Learning Gain is a metric using the assessment scores of the participants. Extracting the Max Score, Pre-assessment, and Post-Assesment scores. The Learning Gain is calculated using a CSC's equation.

$$
    Learning Gain = 100\% * \frac{PostTest Score - PreTest Score}{Maximum Score - PreTest Score}
$$

When the equation returns an undefined answer it is equated to 0%.

Since the Assessment Scores are right after the participant information, the data can be extracted by column coordinates.

In [ ]:
=QUERY(LearningGain,"Select ((Col2-Col1)/(Col3-Col1))*100")

Some of the data in gsheets returns blank since an undefined answer is not returned. To average the score, the unique ids are counted and used as the denominator to the sum of all the participants learning gain.

For the other categories, a dynamic list is created using the columns list created early on. This is then sorted by the use of keywords. 

In [ ]:
Example:
=QUERY(Columns, "Select Col1 WHERE Col1 CONTAINS 'PROGRAM DESIGN'")

The dynamic list is then used as an array in where the individual elements in the dynamic list is used as column names to be extracted and as a added measure it also checks if the rows after the header contains an element or value.

In [ ]:
=QUERY(Filtered!A1:GY, "SELECT "&TEXTJOIN(", ", TRUE, ARRAYFORMULA("Col"&XMATCH(TOCOL(A5:A, 1), Filtered!A1:GY1)))&" WHERE Col1 IS NOT NULL", 1)

Since the count of elements/votes of each subcategory varies, the average score given by the subcategory is given weight by multiplying the average score by the quotient of total count of the votes and the unique id count, turning it into a weighted average.
$$
Weight = \frac{Total Score Count}{Total Unique ID Count}
$$
$$
Weighted Average = {Average Score} * Weight
$$
The sum of the Weighted Average of each subcategory is then divided by the sum of the Total Weight of each subcategory in order to get an accurate Categorical Average. The weighted average in each category is then summed and divided by the total weight of all the subcategories. To show a good approximate of the average of all the scores.

#### Dashboard
The dashboard is a basic scorecard where the relevant is just drawn from the several sheets in the spreadsheet using query. Histogram of the calculated Learning Gain of the Participant is also shown to show where the data skew happens, and to be able to visualize the where the partipants score are placed.

![Dashboard](/Users/internship/Downloads/AllDupe/Dashboard2.png)

## Conclusion and Recommendation

![FlowChart](/Users/internship/Downloads/AllDupe/FlowChart1.png)

The database system hinges on the constant update by the HRMD personel but by automating the calculations and data cleaning the job task can be done with a few clicks. Any other monitoring, i.e. searching for the amount of participants for each department,is easier because all of the data needed is in one place. Any other visualizations can be done easily since the data is automatically sorted within the google sheet and given a metric based on the count of the score versus the count of the participants. 

Since the python code is heavily reliant on lists, there is significant room for optimization if static tuples is used to store the dataframes and the list are deleted. The automation of importing of the AllConcat data to the google sheets was also tried  but the sheer amount of data being written in one instance leads to the API timing out since a free google cloud account is being used. 